In [ ]:
#| default_exp netana_L07

In [ ]:
from graph_tool import Graph
#| hide
from nbdev.showdoc import *

# Lecture 7 - Communities, Clusters, Motifs
* KL finish
* Community finding
* Modularity
* Clustering

## Partitioning KL

$$R = \frac{1}{4}\sum_{i,j}A_{i,j}(1-s_i s_j)$$

With $\sum_{i,j}A_{ij}=\sum_i k_i = \sum_i k_i = \sum_i k_i s_i^2 = \sum_{i,j} k_i \delta_{ij}s_i s_j$ with $k_i$ degree and $\delta_{ij}$ Kronecker delta (1 if equal). But then $R = \frac{1}{4}\sum_{i,j}(k_i \delta_{ij} - A_{ij})s_i s_j$ by substituting in above formula, thus $R = \frac{1}{4}\sum L_{ij} s_i s_j$ where $L$ is the Laplacian matrix $L = D - A$, $D$ diagonal degree matrix.

$$L = $$

In [ ]:
import numpy as np
from graph_tool import Graph

In [ ]:

# L is Laplacian Matrix
# D is Diagonal Degree Matrix



* What is KL-Partitioning?

## Community
#### What is a Community? How is it defined?
Something about the nodes knowing each other. And something about the internal-degree having to be larger than the external-degree. So basically a clique, so a network where everyone knows everyone is a community. It even is a strong community because exactly everyone knows everyone. There can also be a community in a less "optimal" network. So there is a concept of strong and weak communities, depending on the degree of everyone in the group knowing everyone else in the group.
#### Community Detection
Beyond the term Community there is also a mayor problem of detecting communities in networks. This is not easy and not every network does necessarily have a community. So algorithms have to be able to detect them if they are there but also be able to say that the arent. It seems like there are two mayor methods. They are handeled in two subsections, namely Modularity and Divisive, which seem to be just two different methods for doing this.
#### How does the Modularity Method for community detection work?

#### How does the Divisive Method for community detection work?

* What is the process of finding communities in network data?

* What is modularity? How is it defined?

In [ ]:
# What kind of code can we write about community detection? We can probably load any graph and check if there is a community present in it and then get the characteristic metrics for these communities that we can then report on.

def find_community(graph: Graph):
    return {}

def report_communities(communities):
    pass

In [ ]:

G = Graph()

community = find_community(G) # so something like this could be interesting.
report_communities(community) # And then using this community data structure we could maybe report on it.
# Now i definately don't know if this works this way, at least a good guess is that something exists here that we can report on and that we can acutally do in code and probably also usign functions that are already implemented either in graph_tool or networkx.




* What is a cluster? How is it defined?
I think clustering is a whole nother problem again. I could read up on it if there is a similarity or connection between a cluster and a community.



* What is a motif? How is it defined?
I think motif is not part of this lecture maybe the next one, at least i didn't see the word anywhere unitl now.

* I need to get a handle of the formulas, not only to memorize them but to get a pattern recognition working for me, so i can find things again and again. Currently i am only pattern matching. Probably also because the slides dont seem to follow an understandable order. I should rather read the books but i also want to get what he tells us on the slides.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()